In [35]:
import os
import zipfile
import numpy as np
import librosa
import tensorflow as tf
from google.colab import files, drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# ==== STEP 1: UPLOAD DATASET ====

# Choose dataset upload method
UPLOAD_METHOD = "drive"  # Options: "drive" (Google Drive) or "manual" (local upload)

if UPLOAD_METHOD == "drive":
    drive.mount('/content/drive')
    DATASET_PATH = "/content/drive/MyDrive/recordingss"  # Update with actual path
    EXTRACT_PATH = "/content/dataset"

    # Unzip Dataset
    if not os.path.exists(EXTRACT_PATH):
        os.makedirs(EXTRACT_PATH)
    !unzip -q "$DATASET_PATH" -d "$EXTRACT_PATH"
    print("✅ Dataset extracted to", EXTRACT_PATH)

elif UPLOAD_METHOD == "manual":
    print("📂 Upload your dataset ZIP file")
    uploaded = files.upload()
    for filename in uploaded.keys():
        if filename.endswith(".zip"):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall("/content/dataset")
    EXTRACT_PATH = "/content/dataset"
    print("✅ Dataset extracted!")

# Check files
print("📁 Files in dataset:", os.listdir(EXTRACT_PATH))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
unzip:  cannot find or open /content/drive/MyDrive/recordingss, /content/drive/MyDrive/recordingss.zip or /content/drive/MyDrive/recordingss.ZIP.
✅ Dataset extracted to /content/dataset
📁 Files in dataset: []


In [36]:
RECORDINGS_FOLDER ="/content/drive/MyDrive/recordingss"  # Use extracted dataset folder
N_MFCC = 20
MAX_TIME_STEPS = 50

# Function to extract MFCC features
def extract_mfcc(audio_file, max_time_steps=MAX_TIME_STEPS, n_mfcc=N_MFCC):
    y, sr = librosa.load(audio_file, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
  # Ensure consistent shape (20, 50)
    if mfcc.shape[1] < max_time_steps:
        pad_width = max_time_steps - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    elif mfcc.shape[1] > max_time_steps:
        mfcc = mfcc[:, :max_time_steps]

    return mfcc.T  # Shape (max_time_steps, n_mfcc)


In [37]:
# Extract features & labels
X, y = [], []
for audio_file in os.listdir(RECORDINGS_FOLDER):
    if audio_file.endswith('.m4a'):
        label = 1 if "aut_" in audio_file else 0  # Assuming naming convention
        mfcc = extract_mfcc(os.path.join(RECORDINGS_FOLDER, audio_file))
        X.append(mfcc)
        y.append(label)

X = np.array(X)  # Shape: (samples, max_time_steps, n_mfcc)
y = to_categorical(y, num_classes=2)  # Convert labels to categorical


<ipython-input-36-bc003070b4e7>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-36-bc003070b4e7>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-36-bc003070b4e7>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/

In [38]:
# ==== STEP 2: PREPROCESS AUDIO DATA ====
RECORDINGS_FOLDER = EXTRACT_PATH  # Use extracted dataset folder
N_MFCC = 20
MAX_TIME_STEPS = 50


In [39]:
# Function to extract MFCC features
def extract_mfcc(audio_file, max_time_steps=MAX_TIME_STEPS, n_mfcc=N_MFCC):
    y, sr = librosa.load(audio_file, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    # Ensure consistent shape (20, 50)
    if mfcc.shape[1] < max_time_steps:
        pad_width = max_time_steps - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    elif mfcc.shape[1] > max_time_steps:
        mfcc = mfcc[:, :max_time_steps]

    return mfcc.T  # Shape (max_time_steps, n_mfcc)


In [40]:
from sklearn.model_selection import train_test_split

# Check if X and y exist
print(f"✅ Total Dataset Shape: {X.shape}, Labels Shape: {y.shape}")

# Ensure X and y are properly defined before splitting
if 'X' in locals() and 'y' in locals():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"✅ Training Data Shape: {X_train.shape}")
    print(f"✅ Testing Data Shape: {X_test.shape}")
else:
    print("❌ Error: X or y is not defined. Check dataset loading.")


✅ Total Dataset Shape: (43, 50, 20), Labels Shape: (43, 2)
✅ Training Data Shape: (34, 50, 20)
✅ Testing Data Shape: (9, 50, 20)


In [51]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout

# ✅ Define BiLSTM Model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.5, recurrent_dropout=0.5), input_shape=(50, 20)),
    Bidirectional(LSTM(64, dropout=0.5, recurrent_dropout=0.5)),  # Removed one layer
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Output layer for 2 classes
])

# ✅ Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Print Model Summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_9 (Bidirectional)      │ (None, 50, 256)             │         152,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 325,314 (1.24 MB)

 Trainable params: 325,314 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:



# ✅ Train Model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop])


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 457ms/step - accuracy: 0.5000 - loss: 0.7544 - val_accuracy: 0.5556 - val_loss: 0.6713
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 398ms/step - accuracy: 0.5404 - loss: 0.6964 - val_accuracy: 0.5556 - val_loss: 0.6744
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 407ms/step - accuracy: 0.4896 - loss: 0.7692 - val_accuracy: 0.5556 - val_loss: 0.6779
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 428ms/step - accuracy: 0.5496 - loss: 0.6551 - val_accuracy: 0.5556 - val_loss: 0.6773
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step - accuracy: 0.5300 - loss: 0.7346 - val_accuracy: 0.5556 - val_loss: 0.6767
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 770ms/step - accuracy: 0.3799 - loss: 0.7646 - val_accuracy: 0.5556 - val_loss: 0.6760
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 391ms/step - accuracy: 0.4700 - loss: 0.7353 - val_accuracy: 0.5556 - val_loss: 0.6751
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 415ms/step - accuracy: 0.5300 - loss: 0.7147 - val_accuracy: 0.5556 - val_loss:

NameError: name 'history' is not defined

In [31]:
MODEL_FILE = "/content/bilstm_model.h5"
model.save(MODEL_FILE)  # Save model
print(f"✅ Model saved successfully at: {MODEL_FILE}")


✅ Model saved successfully at: /content/bilstm_model.h5


In [33]:
# ==== STEP 4: TEST MODEL WITH NEW AUDIO ====
print("📂 Upload an .m4a file for testing")
uploaded = files.upload()
test_file = list(uploaded.keys())[0]

# Load trained model
model = tf.keras.models.load_model(MODEL_FILE)

# Extract MFCC features from test file
mfcc_features = extract_mfcc(test_file)

# Reshape for LSTM input
mfcc_features = np.expand_dims(mfcc_features, axis=0)  # Shape (1, 50, 20)

# Predict Class
prediction = model.predict(mfcc_features)
predicted_label = "Autistic" if np.argmax(prediction) == 1 else "Non-Autistic"

print(f"🧠 Prediction: {predicted_label}")


📂 Upload an .m4a file for testing


Saving Recording.m4a to Recording.m4a


<ipython-input-22-704faf7ea42c>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
🧠 Prediction: Autistic


NameError: name 'model' is not defined

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved in Google Drive: /content/drive/MyDrive/lstm_model.h5
